### Firms HHI in Smartphone industry

### Data collection & Data wrangling

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

#setting directory
os.chdir("C:/Users/Lfran/OneDrive/Master thesis/Pricing power/Measurement")
share_us = pd.read_csv('0_Data/vendor-US-monthly-201401-202312.csv', encoding='latin1')
share_cn = pd.read_csv('0_Data/vendor-CN-monthly-201401-202312.csv', encoding='latin1')
share_in = pd.read_csv('0_Data/vendor-IN-monthly-201401-202312.csv', encoding='latin1')
share_ww = pd.read_csv('0_Data/vendor-ww-monthly-201401-202312.csv', encoding='latin1')
share_an = pd.read_csv('0_Data/vendor-an-monthly-201401-202412.csv', encoding='latin1')
share_af = pd.read_csv('0_Data/vendor-af-monthly-201401-202412.csv', encoding='latin1')
share_eu = pd.read_csv('0_Data/vendor-eu-monthly-201401-202412.csv', encoding='latin1')
share_oc = pd.read_csv('0_Data/vendor-oc-monthly-201401-202412.csv', encoding='latin1')
share_na = pd.read_csv('0_Data/vendor-na-monthly-201401-202412.csv', encoding='latin1')
share_sa = pd.read_csv('0_Data/vendor-sa-monthly-201401-202412.csv', encoding='latin1')
share_as = pd.read_csv('0_Data/vendor-as-monthly-201401-202412.csv', encoding='latin1')


share_an.columns.tolist()


['Date',
 'Apple',
 'Samsung',
 'Xiaomi',
 'Huawei',
 'Unknown',
 'Sony',
 'HTC',
 'LG',
 'OnePlus',
 'Lenovo',
 'Oppo',
 'Vivo',
 'Google',
 'Motorola',
 'Archos',
 'ZTE',
 'Nokia',
 'Asus',
 'Alcatel',
 'Realme',
 'Honor',
 'Acer',
 'Cubot',
 'Meizu',
 'Infinix',
 'Itel',
 'bq',
 'Vodafone',
 'Karbonn',
 'Tecno',
 'Yu',
 'Blu',
 'Other']

In [32]:

#converting format for data to year
# Correct assignment for each frame
import pandas as pd

# List of (DataFrame, Region Label) pairs
region_data = [
    (share_us, 'US'),
    (share_cn, 'China'),
    (share_in, 'India'),
    (share_ww, 'Worldwide'),
    (share_as, 'Asia'),
    (share_eu, 'Europe'),
    (share_af, 'Africa'),
    (share_na, 'North America'),
    (share_sa, 'South America'),
]

# Process each region
processed_regions = []

for df, region in region_data:
    # Ensure 'Date' is named correctly
    df.rename(columns={df.columns[0]: 'Date'}, inplace=True)

    # Convert Date to datetime and extract year
    df['Date'] = pd.to_datetime(df['Date'].astype(str), format='%Y-%m', errors='coerce')
    df['Year'] = df['Date'].dt.year

    # Group by Year and average monthly shares
    df_yearly = df.drop(columns='Date').groupby('Year', as_index=False).mean()

    # Melt to long format
    df_long = df_yearly.melt(
        id_vars='Year',
        var_name='Company Name',
        value_name=f'Market Share {region}'
    )

    processed_regions.append(df_long)

# Optionally: merge all into one big table on ['Year', 'Company Name']
from functools import reduce
market_share_all = reduce(
    lambda left, right: pd.merge(left, right, on=['Year', 'Company Name'], how='outer'),
    processed_regions
)

# Done: `market_share_all` has average yearly market share for all regions
market_share_all.head()



,Year,Company Name,Market Share US,Market Share China,Market Share India,Market Share Worldwide,Market Share Asia,Market Share Europe,Market Share Africa,Market Share North America,Market Share South America
0,2014,360,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,AIS,NaN,NaN,NaN,0.0000,0.000000,NaN,NaN,NaN,NaN
2,2014,AT&T,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
3,2014,Acer,NaN,NaN,NaN,0.0850,0.054167,0.239167,0.039167,0.042500,0.028333
4,2014,Alcatel,0.460833,0.22,0.328333,0.5025,0.267500,0.553333,0.828333,0.849167,0.623333


### Calculating HHI per region and year

In [33]:
# List all region-specific market share columns
region_columns = [col for col in market_share_all.columns if col.startswith('Market Share')]

# Create a dict to hold the HHI DataFrames for each region
hhi_by_region = {}

for region_col in region_columns:
    region = region_col.split()[-1]  # extract "US", "CN", etc.
    
    # Drop NaNs and compute HHI per year
    hhi = (
        market_share_all
        .dropna(subset=[region_col])
        .groupby('Year')[region_col]
        .apply(lambda x: (x ** 2).sum())
        .reset_index()
        .rename(columns={region_col: f'HHI {region}'})
    )
    
    hhi_by_region[region] = hhi

# Merge all HHI tables into one
from functools import reduce
hhi_all = reduce(
    lambda left, right: pd.merge(left, right, on='Year', how='outer'),
    hhi_by_region.values()
)

# View final HHI table
hhi_all


,Year,HHI US,HHI China,HHI India,HHI Worldwide,HHI Asia,HHI Europe,HHI Africa,HHI America
0,2014,3428.077370,1456.549042,2413.892792,1932.607548,2010.891799,2388.405499,2088.826903,2107.717779
1,2015,3363.484723,1353.921085,1876.545974,1721.022413,1699.035642,2119.853848,2002.555172,1900.859551
2,2016,3614.940963,1444.770395,1690.272047,1728.244368,1677.972778,2144.307907,2264.252159,2004.766710
3,2017,3685.550994,1379.739306,1121.783083,1621.827501,1469.067359,2205.892159,1564.763535,2397.266169
4,2018,3699.374272,1423.895593,1178.415462,1549.312431,1362.241590,2137.609417,1492.502363,2497.837162
5,2019,3770.626702,1364.297402,1342.802761,1723.881441,1480.523268,2261.667553,1668.790263,2717.248514
6,2020,4196.530239,1640.479337,1462.759575,1858.702981,1522.507019,2321.701369,1747.011124,2712.722447
7,2021,4128.727434,2195.304306,1561.391513,1779.146754,1396.449482,2376.009281,1655.367149,2630.389235
8,2022,4092.984210,1898.742094,1544.636431,1830.602328,1446.642871,2386.644047,1650.426833,2534.200948
9,2023,4123.768990,1852.896461,1406.151209,1746.050975,1340.006690,2422.347655,1575.345382,2325.642692


### Calculating Firm's HHi per firm and year

In [40]:
import pandas as pd

# 1. Identify all HHI‐contribution columns
hhi_cols = [c for c in market_share_all.columns if c.startswith('HHI Contribution')]

# 2. Melt into long format
firm_hhi = market_share_all.melt(
    id_vars=['Year', 'Company Name'],
    value_vars=hhi_cols,
    var_name='Region',
    value_name='HHI Contribution'
)

# 3. Clean up the Region names (strip the prefix)
firm_hhi['Region'] = firm_hhi['Region'].str.replace('HHI Contribution ', '')

# 4. (Optional) Sort for readability
firm_hhi = firm_hhi.sort_values(['Region', 'Year', 'HHI Contribution'], ascending=[True, True, False])

# Classification function
def classify_market_power(hhi):
    if hhi >= 2500:
        return "High market power"
    elif hhi >= 1500:
        return "Moderate market power"
    elif hhi > 0:
        return "Low market power"
    else:
        return "No market power"

# Apply to your existing firm_hhi DataFrame
firm_hhi['Market Power Class'] = firm_hhi['HHI Contribution'].apply(classify_market_power)

# 5. Inspect your new table
firm_hhi.head( ) # Check the columns


,Year,Company Name,Region,HHI Contribution,Market Power Class
7528,2014,Nokia,Africa,991.987517,Low market power
7548,2014,Samsung,Africa,619.304701,Low market power
7564,2014,Unknown,Africa,358.534225,Low market power
7542,2014,RIM,Africa,72.476844,Low market power
7470,2014,Apple,Africa,27.057336,Low market power


In [48]:
def get_region_hhi_table(region_code):
    """
    Returns a DataFrame of Year × Company with HHI contribution and market‐power class
    for the given region code (e.g. 'US', 'CN', 'IN', etc.).
    """
    # Filter to the chosen region
    df_region = firm_hhi[firm_hhi['Region'] == region_code].copy()
    
    # Select and sort
    df_region = df_region[
        ['Year', 'Company Name', 'HHI Contribution', 'Market Power Class']
    ].sort_values(['Year', 'HHI Contribution'], ascending=[True, False])
    
    return df_region

# — Example usage —
us_hhi = get_region_hhi_table('US')
china_hhi = get_region_hhi_table('China')
europe_hhi = get_region_hhi_table('Europe')
north_america_hhi = get_region_hhi_table('North America')
africa_hhi = get_region_hhi_table('Africa')
asia_hhi = get_region_hhi_table('Asia')
oceania_hhi = get_region_hhi_table('Oceania')
antarctica_hhi = get_region_hhi_table('Antarctica')
india_hhi = get_region_hhi_table('India')
south_america_hhi = get_region_hhi_table('South America')
print(china_hhi.head(20))

firm_hhi.to_csv('0_Data/generated_csv/firm_hhi.csv', index=False)

      Year Company Name  HHI Contribution Market Power Class
1250  2014        Apple        508.389756   Low market power
1344  2014      Unknown        399.067211   Low market power
1328  2014      Samsung        345.402225   Low market power
1351  2014       Xiaomi         86.862400   Low market power
1277  2014       Huawei         71.995225   Low market power
1293  2014       Lenovo         11.571336   Low market power
1274  2014          HTC          9.838678   Low market power
1267  2014      Coolpad          8.875434   Low market power
1354  2014          ZTE          4.572469   Low market power
1308  2014        Nokia          3.900625   Low market power
1332  2014         Sony          1.564584   Low market power
1297  2014        Meizu          1.071225   Low market power
1311  2014         Oppo          1.059184   Low market power
1255  2014          BBK          1.020100   Low market power
1272  2014       Google          0.329667   Low market power
1289  2014           LG 

### Concentration Ration CRn

In [ ]:
import pandas as pd

def compute_cr(market_share_all: pd.DataFrame, n: int = 4) -> pd.DataFrame:
    """
    Compute CRₙ (sum of the top n market shares) per Year and Region.
    
    market_share_all must have:
      - 'Year'
      - 'Company Name'
      - 'Market Share XX' columns for each region (e.g. 'Market Share US', etc.)
    
    Returns a DataFrame with columns ['Year','Region',f'CR{n}'].
    """
    # 1. Melt to long form
    ms_long = market_share_all.melt(
        id_vars=['Year','Company Name'],
        value_vars=[c for c in market_share_all.columns if c.startswith('Market Share')],
        var_name='Region',
        value_name='Market Share'
    )
    ms_long['Region'] = ms_long['Region'].str.replace('Market Share ', '', regex=False)
    
    # 2. Group & compute CRₙ
    cr = (
        ms_long
        .groupby(['Year','Region'], as_index=False)['Market Share']
        .apply(lambda shares: shares.nlargest(n).sum())
        .rename(columns={'Market Share': f'CR{n}'})
    )
    
    return cr

# Example: 4-firm concentration
smartphone_cr4 = compute_cr(market_share_all, n=4)


# Example: 8-firm concentration
smartphone_cr8 = compute_cr(market_share_all, n=8)


smartphone_cr4.to_csv('firm_cr4.csv', index=False)
smartphone_cr8.to_csv('firm_cr8.csv', index=False)

   Year  Region        CR4
0  2014  Africa  83.830000
1  2014    Asia  81.395000
2  2014   China  70.429167
3  2014  Europe  78.943333
4  2014   India  90.238333
   Year  Region        CR8
0  2014  Africa  95.895833
1  2014    Asia  92.272500
2  2014   China  88.431667
3  2014  Europe  94.218333
4  2014   India  96.542500
